In [1]:
# %load test.py
import torch
import argparse
import pretrainedmodels
import torch.nn as nn

from torchvision import datasets
from utils import *
from collections import OrderedDict
from torch.autograd import Variable
import torchvision.models as models

parser = argparse.ArgumentParser('Transfer')
parser.add_argument('--exp_name', type=str,
                    default='transfer', help='search model name')
parser.add_argument('--model_name', type=str,
                    default='xception', help='search model name')
parser.add_argument('--classes', type=int, default=9, help='num of MB_layers')
parser.add_argument('--batch_size', type=int, default=64, help='batch size')
parser.add_argument('--epochs', type=int, default=10, help='num of epochs')
parser.add_argument('--seed', type=int, default=0, help='seed')
parser.add_argument('--learning_rate', type=float,
                    default=0.01, help='initial learning rate')
parser.add_argument('--learning_rate_min', type=float,
                    default=1e-8, help='min learning rate')
parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
parser.add_argument('--weight_decay', type=float,
                    default=3e-4, help='weight decay')
parser.add_argument('--train_interval', type=int, default=1,
                    help='train to print frequency')
parser.add_argument('--val_interval', type=int, default=5,
                    help='evaluate and save frequency')
parser.add_argument('--smooth', type=float,
                    default=0.0, help='label smoothing')
parser.add_argument('--dropout_rate', type=float,
                    default=0.5, help='drop out rate')
parser.add_argument('--drop_path_prob', type=float,
                    default=0.3, help='drop_path_prob')
parser.add_argument('--grad_clip', type=float,
                    default=5, help='gradient clipping')
parser.add_argument('--gpu', type=int, default=0, help='gpu id')
parser.add_argument('--resume', type=bool, default=False, help='resume')
# ******************************* dataset *******************************#
parser.add_argument('--dataset', type=str, default='imagenet')
parser.add_argument('--data_dir', type=str, default='./valAndTrain')
parser.add_argument('--cutout', action='store_true',
                    default=False, help='use cutout')
parser.add_argument('--cutout_length', type=int,
                    default=16, help='cutout length')

args = parser.parse_args(args=[])


def extract_feature(model, datas, pre_trained_weights_path, device, feature_layer="bn4"):
    if feature_layer not in ["bn4", "conv4"]:
        raise ValueError(
            "feature_layer must be one of `bn4`、`conv4` !")

    # rebuild pre-trained model for feature extraction
    new_model = OrderedDict()
    for name, module in net._modules.items():
        new_model[name] = module
        if name == feature_layer:
            break
    premodel = nn.Sequential(new_model)
    premodel.eval()
    premodel.to(device)
    print("Model rebuilded!")

    # reload pre-trained weights
    model_dict = premodel.state_dict()
    pretrianed_dict = torch.load(
        pre_trained_weights_path, map_location=torch.device('cpu'))
    pretrianed_dict = {k: v for k, v in pretrianed_dict[
        'supernet_state'].items() if k in model_dict.keys()}

    model_dict.update(pretrianed_dict)
    premodel.load_state_dict(model_dict)
    print("Pre-trained weights loaded!")

    output = []
    print("Start extraction!")
    with torch.no_grad():
        for step, (inputs, targets) in enumerate(datas):
            inputs, targets = inputs.to(device), targets.to(device)
            result = premodel(inputs)
            # res = result.numpy()
            output.append(result)
            del result
            print("Step: {} Finished!".format(step))
    return np.concatenate(output, axis=0)


if __name__ == "__main__":
    if not torch.cuda.is_available():
        device = torch.device('cpu')
    else:
        torch.cuda.set_device(args.gpu)
#         cudnn.benchmark = True
#         cudnn.enabled = True
        device = torch.device("cuda")

    # 载入数据集
    train_transform, valid_transform = data_transforms(args)
    # 传给root的参数是你存放数据的文件夹
    val_data = datasets.ImageFolder(root=r'C:/mushroomdata/512test/valAndTrain/val1', transform=valid_transform)
    print(val_data.classes)  #根据分的文件夹的名字来确定的类别
    print(val_data.class_to_idx) #按顺序为这些类别定义索引为0,1...
    print(val_data.imgs) #返回从所有文件夹中得到的图片的路径以及其类别
    valid_loader = torch.utils.data.DataLoader(val_data, batch_size=args.batch_size,
                                               shuffle=False)
    print(valid_loader)
    # path是你预训练好的模型权重
    path = r'C:/mushroomdata/xception_transfer_states.pt.tar'
    # net是读取xception模型
    net = pretrainedmodels.__dict__["xception"](pretrained='imagenet')
#     net = pretrainedmodels.__dict__["xception"](pretrained==False)\
#     net = models.xception(pretrained=False)  
#     net.load_state_dict(torch.load(r'C:/mushroomdata/xception-43020ad28.pth.pth'))
    # ipt是输入的数据，这里你需要自己替换成你自己的数据
    # ipt = Variable(torch.rand(8, 3, 224, 224))

    # 提取特征，feature_layer选择bn4或者conv4，就是相应层的特征
    feature = extract_feature(net, valid_loader, path, device, feature_layer="bn4")
    print(feature.shape)
#     print(feature)


['Agaricus', 'Hygrocybe', 'Suillus']
{'Agaricus': 0, 'Hygrocybe': 1, 'Suillus': 2}
[('C:/mushroomdata/512test/valAndTrain/val1\\Agaricus\\190_qDzeyg_X_Ek.jpg', 0), ('C:/mushroomdata/512test/valAndTrain/val1\\Agaricus\\275_v_WFr722NZw.jpg', 0), ('C:/mushroomdata/512test/valAndTrain/val1\\Agaricus\\281_aj5i2CFRn3A.jpg', 0), ('C:/mushroomdata/512test/valAndTrain/val1\\Agaricus\\402_8-goocGqseI.jpg', 0), ('C:/mushroomdata/512test/valAndTrain/val1\\Agaricus\\415_sdgtnIQ3Dxw.jpg', 0), ('C:/mushroomdata/512test/valAndTrain/val1\\Agaricus\\428_KartCMfFQYk.jpg', 0), ('C:/mushroomdata/512test/valAndTrain/val1\\Hygrocybe\\032_DZ1xy6htm0M.jpg', 1), ('C:/mushroomdata/512test/valAndTrain/val1\\Hygrocybe\\049_XD-_-BeBBR4.jpg', 1), ('C:/mushroomdata/512test/valAndTrain/val1\\Hygrocybe\\071_JRMa8LXarFk.jpg', 1), ('C:/mushroomdata/512test/valAndTrain/val1\\Hygrocybe\\109_HMXraCYzkTA.jpg', 1), ('C:/mushroomdata/512test/valAndTrain/val1\\Hygrocybe\\133_3_ieL2NtOF0.jpg', 1), ('C:/mushroomdata/512test/valAn

In [2]:
print(feature)

[[[[ 3.25362039e+00  4.91956520e+00  4.72162151e+00 ...  5.05856609e+00
     5.66771078e+00  3.79015660e+00]
   [ 7.56452179e+00  1.18196087e+01  1.20729113e+01 ...  1.23328352e+01
     1.23970423e+01  7.97110510e+00]
   [ 1.00914860e+01  1.58868198e+01  1.63203278e+01 ...  1.58289347e+01
     1.52423086e+01  9.65431976e+00]
   ...
   [ 8.62924767e+00  1.40669060e+01  1.45223532e+01 ...  1.26763477e+01
     1.18179016e+01  7.66791582e+00]
   [ 6.56880903e+00  1.08691711e+01  1.15809336e+01 ...  1.02339344e+01
     9.53787994e+00  6.40585566e+00]
   [ 3.35072684e+00  5.49004030e+00  6.10802794e+00 ...  5.32990026e+00
     4.87290812e+00  3.39277434e+00]]

  [[ 2.61842537e+00  4.01375389e+00  4.39132452e+00 ...  6.86800528e+00
     6.72058296e+00  5.01136637e+00]
   [ 6.53183651e+00  1.02060480e+01  1.15801945e+01 ...  1.52660475e+01
     1.40714397e+01  9.93261051e+00]
   [ 8.86711788e+00  1.39366808e+01  1.56740761e+01 ...  1.86612663e+01
     1.68790627e+01  1.15955029e+01]
   ...
   

In [3]:
feature = np.array(feature）
print("Training feature dimension: ", feature.shape)

SyntaxError: invalid character in identifier (<ipython-input-3-a1660b6ccf62>, line 1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import ImageFile,Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

# PCA2D
pca = PCA(n_components=2)
training_feature_pca_2D = pca.fit_transform(training_feature)
# test_feature_pca_2D = pca.transform(test_feature)

In [ ]:
#pca 2d
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(1, 1, 1)
colors = ['r', 'g', 'b','r', 'g', 'c','r', 'g', 'b']
markers = ['o', 'P', 'v','o', 'P', 'v','o', 'P', 'v']
classes = np.sort(np.unique(train_label))
labels = ['Agaricus', 'Amanita', 'Boletus', 'Cortinarius', 'Entoloma', 'Hygrocybe', 'Lactarius', 'Russula', 'Suillus']
for class_ix, marker, color, label in zip(classes, markers, colors, labels):
     ax.scatter(training_feature_pca_2D[np.where(train_label == class_ix), 0],
             training_feature_pca_2D[np.where(train_label == class_ix), 1],
             marker=marker, color=color,
             linewidth='1', alpha=0.9, label=label)  
#          ax.legend(loc='best')
plt.xlabel('First principal component')
plt.ylabel('Second Principal Component')
plt.title("2 component PCA")
plt.show()

In [ ]:
model = SVC(kernel ='linear', C = 1)

In [ ]:
# 2D Training
from numpy import array
# print（map(list, zip(*training_feature_pca_2D))）
print(array(training_feature_pca_2D))
# print(training_feature_pca_2D.tolist())
print(train_label)
print(np.array(train_label.transpose()))
model.fit(training_feature_pca_2D, train_label)

# Evaluation
y_pred = model.predict(test_feature_pca_2D)

# Result
acc = accuracy_score(test_label, y_pred)
print("Accuracy: {:.2f}".format(acc*100))